Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Deployment sample (under construction)
* Setup Workspace
* Register a trained model
* Convert the model to DLC for camera
* Build IoT Image 
* Create Hub and Prepare for device and deploy
* Deploy Model to Camera

![title](aml_config/AMLimages/VAIDKe2e.png)

In [ ]:
# For prod
!pip install azureml-core azureml-contrib-iot azure-mgmt-containerregistry azure-cli
!az extension add --name azure-cli-iot-ext

In [ ]:
import os
print(os.__file__)

In [ ]:
# Check core SDK version number
import azureml.core as azcore

print("SDK version:", azcore.VERSION)

### Create a Workspace
#### Change this cell from markdown to code and run this if you need to create a workspace 
#### Update the values for your workspace below
ws=Workspace.create(subscription_id="replace-with-subscription-id",
                resource_group="your-resource-group",
                name="your-workspace-name",
                location="eastus2")
                
ws.write_config()

In [ ]:
# Initialize existing Workspace 
# reading config from ./aml_config/config.json
# update this file with your subscription ID, AML workspace and resource group

from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
# set the location for retrained model
# MUST include va-snpe-library_config.json file
retrained_model_location = '<path to the location of your trained model>'

![title](aml_config/AMLimages/Model.png)

In [ ]:
# Register the model by uploading the model file and label file

from azureml.core.model import Model

model = Model.register(model_path = retrained_model_location,
                      model_name = "name_of_your_model",
                      tags = {"data": "Imagenet", "model": "object_classification", "type": "imagenet"},
                      description = "Description_of_your_model",
                      workspace = ws)

![title](aml_config/AMLimages/Image.png)

In [ ]:
from azureml.contrib.iot.model_converters import SnpeConverter

# submit a compile request to convert the model to SNPE compatible DLC file
compile_request = SnpeConverter.convert_tf_model(
    ws,
    source_model=model,
    input_node="input",
    input_dims="1,224,224,3",
    outputs_nodes = ["final_result"],
    allow_unconsumed_nodes = True)
print(compile_request._operation_id)

In [ ]:
# wait for the request to complete
compile_request.wait_for_completion(show_output=True)

In [ ]:
# Get converted model
converted_model = compile_request.result
print(converted_model.name, converted_model.url, converted_model.version, converted_model.id, converted_model.created_time)

In [ ]:
# set the image name. must be lower case alphanumeric characters, '-' or '.'
image_name = "name_your_image"+str(model.version)
print("image name:", image_name)

In [ ]:
from azureml.core.image import Image
from azureml.contrib.iot import IotContainerImage

# create the IoT container image for the camera - Yocto/ARM32.
# includes scripts to access the VAM engine to send camer frames for inferecning
print ('We will create an image for you now ...')
image_config = IotContainerImage.image_configuration(
                                 architecture="arm32v7",
                                 execution_script="main.py", 
                                 dependencies=["camera.py","iot.py","ipcprovider.py","utility.py", "frame_iterators.py"],
                                 docker_file="Dockerfile", 
                                 tags = ["mobilenet"],
                                 description = "MobileNet model retrained soda cans")

image = Image.create(name = image_name,
                     models = [converted_model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output = True)

![title](aml_config/AMLimages/Deploy.png)

In [ ]:
# Parameter list to configure the IoT Hub and the IoT Edge device

# Pick a name for what you want to call the module you deploy to the camera
module_name = "name of the AI module"++

# Resource group in Azure 
resource_group_name= ws.resource_group
iot_rg=resource_group_name

# Azure region where your services will be provisioned
iot_location="location"

# Azure IoT Hub name
iot_hub_name="name of IoT Hub"

# Pick a name for your camera
iot_device_id="name of IoT Edge Device"

# Pick a name for the deployment configuration
iot_deployment_id="name for the deployment ID"

In [ ]:
# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(image.image_location), "<-- this is the URI configured in the IoT Hub for the device")
print('{}'.format(reg_name))
print('{}'.format(subscription_id))
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group_name, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value

### Deployment file
This is the deployment.json file that you will use to deploy your model.  Please see the other sample notebooks on using this file to deploy the new model you created.

In [ ]:
file = open('./deployment-template.json')
contents = file.read()
contents = contents.replace('__MODULE_NAME', module_name)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
!az login 
!az account set --subscription "<subscription>"

In [ ]:
print("Pushing deployment to IoT Edge device")

In [ ]:
print ("Set the deployement") 
!az iot edge set-modules --device-id $iot_device_id --hub-name $iot_hub_name --content deployment.json

In [ ]:
# THE END
# now check the IoT Edge device in Azure portal to verify 
# the URI is updated and will be pushed to the device when it comes online.